In [7]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
# 设置中文字体为宋体
plt.rcParams['font.sans-serif'] = ['SimHei']
# 设置英文字体为新罗马
plt.rcParams['font.serif'] = ['Times New Roman']
# 字体大小
plt.rcParams['font.size'] = 12  
# 正常显示负号
plt.rcParams['axes.unicode_minus'] = False 

In [9]:
df1 = pd.read_csv('中间数据集\\按品类的日粒度时序数据集\\花菜类.csv')
df2 = pd.read_csv('中间数据集\\按品类的日粒度时序数据集\\花叶类.csv')
df3 = pd.read_csv('中间数据集\\按品类的日粒度时序数据集\\茄类.csv')
df4 = pd.read_csv('中间数据集\\按品类的日粒度时序数据集\\辣椒类.csv')
df5 = pd.read_csv('中间数据集\\按品类的日粒度时序数据集\\食用菌.csv')
df6 = pd.read_csv('中间数据集\\按品类的日粒度时序数据集\\水生根茎类.csv')

# 确定约束条件：时间序列的置信度


置信区间是一个统计概念，用于表示对于一个参数或统计量的估计值的不确定性范围。它并不直接表示预测值的范围，而是关于估计值的一个区间，使得我们可以合理地估计真实值可能位于该区间内的概率。

具体来说，如果我们计算了一个参数的置信区间，例如均值的置信区间，那么该区间表示了我们对真实均值所持的一定程度的信心。例如，95%的置信区间意味着在多次重复抽样的情况下，我们预计约有95%的情况下，真实均值会落在该区间内。这并不意味着在任何特定情况下预测值都会落在这个区间内，因为每个样本可能都会有一定的随机性。

置信区间通常与统计假设检验相关，它们用于帮助我们评估样本数据中的估计值（如均值、方差等）的可靠性，并提供了一种衡量估计的不确定性的方式。更广泛地说，置信区间是统计推断中的一个重要工具，用于从样本数据中获取关于总体参数的信息。

总之，置信区间表示了对于估计值的不确定性，并告诉我们在一定置信水平下真实值可能位于该区间内。但它并不直接表示预测值的范围，因为预测值通常还涉及到更多的模型和假设。

<br><br><br><br><br>

确定时间序列的置信区间通常涉及统计分析方法，其中最常见的方法之一是使用基于统计模型的方法，例如ARIMA（自回归集成滑动平均）、GARCH（广义自回归条件异方差）等。下面是一个简单的方法，使用移动平均和标准差来估计时间序列的置信区间：

计算移动平均：首先，计算时间序列的移动平均。移动平均可以帮助平滑时间序列，减少噪音的影响。你可以选择不同的窗口大小，例如3天、7天或30天的移动平均，具体取决于你的数据和需求。

计算标准差：接下来，计算时间序列的标准差。标准差衡量了数据点的离散程度，较大的标准差表示数据波动较大，较小的标准差表示数据波动较小。

计算置信区间：一旦你有了移动平均和标准差，你可以使用统计原理来计算置信区间。通常情况下，你可以使用正态分布或t分布来估计置信区间，具体取决于你的样本大小和数据的性质。常见的置信水平包括95%和99%。

对于正态分布，你可以使用以下公式来计算置信区间：
置信区间 = 移动平均 ± （Z值 * 标准差 / sqrt(样本大小)）
其中，Z值是正态分布对应于所选置信水平的分位数，例如95%置信水平对应的Z值约为1.96。

对于t分布，你可以使用以下公式来计算置信区间：
置信区间 = 移动平均 ± （t值 * 标准差 / sqrt(样本大小)）
其中，t值是t分布对应于所选置信水平和自由度的分位数。

请注意，这只是一个简单的方法，适用于某些情况下。如果你的时间序列具有复杂的季节性、趋势或异方差性质，或者你需要更高级的置信区间估计方法，那么可能需要使用更复杂的统计模型，或者考虑使用专业的时间序列分析工具和软件。此外，选择合适的置信水平也要考虑你的具体需求和风险偏好。

In [10]:
import numpy as np
from scipy.stats import norm, t


def find_bound(df, name):
    # 计算移动平均和标准差（这里以7天的窗口为例）
    window_size = 7
    moving_avg = np.convolve(df[name], np.ones(window_size)/window_size, mode='valid')
    std_dev = np.std(df[name])

    # 选择置信水平（例如，95%）
    confidence_level = 0.6

    # 计算置信区间的半宽度
    if len(df[name]) >= 30:
        # 使用正态分布
        z_value = norm.ppf((1 + confidence_level) / 2)
        margin_error = z_value * (std_dev / np.sqrt(len(df[name])))
    else:
        # 使用t分布，如果样本大小较小
        degrees_of_freedom = len(df[name]) - 1
        t_value = t.ppf((1 + confidence_level) / 2, df=degrees_of_freedom)
        margin_error = t_value * (std_dev / np.sqrt(len(df[name])))

    # 计算置信区间
    lower_bound = min(moving_avg - margin_error)
    upper_bound = max(moving_avg + margin_error)

    return lower_bound, upper_bound

lower_bound_1_1, upper_bound_1_1 = find_bound(df1, '销量(千克)')
lower_bound_1_2, upper_bound_1_2 = find_bound(df1, '成本加成定价')

lower_bound_2_1, upper_bound_2_1 = find_bound(df2, '销量(千克)')
lower_bound_2_2, upper_bound_2_2 = find_bound(df2, '成本加成定价')

lower_bound_3_1, upper_bound_3_1 = find_bound(df3, '销量(千克)')
lower_bound_3_2, upper_bound_3_2 = find_bound(df3, '成本加成定价')

lower_bound_4_1, upper_bound_4_1 = find_bound(df4, '销量(千克)')
lower_bound_4_2, upper_bound_4_2 = find_bound(df4, '成本加成定价')

lower_bound_5_1, upper_bound_5_1 = find_bound(df5, '销量(千克)')
lower_bound_5_2, upper_bound_5_2 = find_bound(df5, '成本加成定价')

lower_bound_6_1, upper_bound_6_1 = find_bound(df6, '销量(千克)')
lower_bound_6_2, upper_bound_6_2 = find_bound(df6, '成本加成定价')

In [11]:
result = pd.DataFrame({
                '品类': ['花菜类', '花叶类', '茄类', '辣椒类', '食用菌', '水生根茎类'],
                '销量(千克)_下界': [lower_bound_1_1, lower_bound_2_1, lower_bound_3_1, lower_bound_4_1, lower_bound_5_1, lower_bound_6_1],
                '销量(千克)_上界': [upper_bound_1_1, upper_bound_2_1, upper_bound_3_1, upper_bound_4_1, upper_bound_5_1, upper_bound_6_1],
                '成本加成定价_下界': [lower_bound_1_2, lower_bound_2_2, lower_bound_3_2, lower_bound_4_2, lower_bound_5_2, lower_bound_6_2],
                '成本加成定价_上界': [upper_bound_1_2, upper_bound_2_2, upper_bound_3_2, upper_bound_4_2, upper_bound_5_2, upper_bound_6_2]
                })
result

,品类,销量(千克)_下界,销量(千克)_上界,成本加成定价_下界,成本加成定价_上界
0,花菜类,2.493046,115.753382,3.718730,22.908976
1,花叶类,13.864352,471.022934,2.132901,10.954627
2,茄类,-0.344874,57.698732,4.173864,17.451418
3,辣椒类,3.853574,257.681711,2.470155,28.757278
4,食用菌,1.602830,250.875884,6.421218,18.767597
5,水生根茎类,1.225343,130.450371,5.753121,24.441389


In [12]:
result.to_excel('中间数据集\\2_4_各品类对销量与成本加成定价的约束.xlsx', index=False)